In [1]:
# author: Yin 
#4 27 2020, change panel identifier from id to csid, get identical results as LG
# RRM with panel data
# Three alternatives: sq,a,b; 

#further test with different mu values mu=10,or mu=1, mu=0.1, and test with covariates

In [2]:
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import *


In [3]:
#Calculating confidence intervals for elasticities requires interval arithmetics 
#pip install python-intervals in anaconda prompt
#in case python-intervals doesn't work, also install a newer version "portion"

import portion as p
import intervals as ivs
import interval as iv

In [4]:
print("Running 03logit_rrm_0427.py...")

Running 03logit_rrm_0427.py...


In [5]:
# Read the data
df = pd.read_csv("agree0623_all_0427.csv")
database = db.Database("agree0623_all_0427",df)
# They are organized as panel data. The variable ID identifies each individual.
database.panel("csid")
# use the names of the variable as Python variable.
globals().update(database.variables)

In [6]:
# Normalize the weights
sumWeight = database.data['weight'].sum()
print(sumWeight) 
##result=7423.735097, sample size=7414

7423.735097


In [7]:
normalizedWeight = weight * 7414 / 7423.735097
# originally weight the whole sample use 1123 cases, 
# then delete nonresponse for choice questions, and 945 cases are used for logit

In [8]:
# Parameters to be estimated 
#NZone variables are already scaled: zone/100
#use 0427 biogeme rum estimates as starting values 
B_NZONE1 = Beta('B_NZONE1',-0.0682,None,None,0)
B_NZONE2 = Beta('B_NZONE2',-0.0372,None,None,0)
B_NZONE3 = Beta('B_NZONE3',-0.0283,None,None,0)
B_TOWNDOWN = Beta('B_TOWNDOWN',0.0273,None,None,0)
B_TOWNMID = Beta('B_TOWNMID',0.114,None,None,0)
B_TOWNUP = Beta('B_TOWNUP',0.205,None,None,0)
B_ECO = Beta('B_ECO',0.053,None,None,0)
B_REC = Beta('B_REC',0.00895,None,None,0)
B_DRY = Beta('B_DRY',-0.215,None,None,0)
B_TAX = Beta('B_TAX',-0.0257,None,None,0)
# mu
mu    = Beta('mu',5.51,0,10,0)

In [9]:
# covariates, starting values from rum scale 04 27 2020
B_ASCEND = Beta('B_ASCEND',0.346,None,None,0)
B_DESCEND = Beta('B_DESCEND',0.429,None,None,0)
B_INCPP50K = Beta('B_INCPP50K',0.526,None,None,0)
B_INCPP75K = Beta('B_INCPP75K',0.912,None,None,0)
B_INCPP75KM = Beta('B_INCPP75KM',0.485,None,None,0)
B_F1FLOW = Beta('B_F1FLOW',-0.469,None,None,0)
B_F3PROTUP = Beta('B_F3PROTUP',-0.00267,None,None,0)
B_F4CTH2O = Beta('B_F4CTH2O',-0.243,None,None,0)
B_TAXGRT = Beta('B_TAXGRT',0.373,None,None,0)


In [10]:
# Definition of the utility functions
scale=B_ASCEND*ascend+B_DESCEND*descend+B_INCPP50K*incpp_50k+B_INCPP75K*incpp_75k+B_INCPP75KM*incpp_75km+B_F1FLOW*f1_flow+B_F3PROTUP*f3_protup+B_F4CTH2O*f4_CTh2o+B_TAXGRT*taxgrt


RSQ =scale* mu * (log( 1 + exp((B_NZONE1/mu) * (nzone1_a-nzone1_sq))) + log(1 + exp((B_NZONE1/mu ) * (nzone1_b-nzone1_sq))) + log(1 + exp((B_NZONE2/mu) * (nzone2_a-nzone2_sq))) + log(1 + exp((B_NZONE2 / mu ) * (nzone2_b-nzone2_sq))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_a-nzone3_sq))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_b-nzone3_sq))) + log( 1 + exp((B_TOWNDOWN / mu ) * (towndown_a-towndown_sq))) + log(1 + exp((B_TOWNDOWN / mu ) * (towndown_b-towndown_sq)))  + log(1 + exp((B_TOWNMID / mu ) * (townmid_a-townmid_sq))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_b-townmid_sq))) + log(1 + exp((B_TOWNUP / mu ) * (townup_a-townup_sq))) + log(1 + exp((B_TOWNUP / mu ) * (townup_b-townup_sq))) + log( 1 + exp((B_ECO / mu ) * (eco_a-eco_sq))) + log(1 + exp((B_ECO / mu ) * (eco_b-eco_sq))) + log(1 + exp((B_REC / mu ) * (rec_a-rec_sq))) + log(1 + exp((B_REC / mu ) * (rec_b-rec_sq))) + log(1 + exp((B_DRY / mu ) * (dry_a-dry_sq))) + log(1 + exp((B_DRY / mu ) * (dry_b-dry_sq))) + log(1 + exp((B_TAX / mu ) * (tax_a-tax_sq))) + log(1 + exp((B_TAX / mu ) * (tax_b-tax_sq))))                                                
          
RA  =scale* mu * (log( 1 + exp((B_NZONE1/mu) * (nzone1_sq-nzone1_a))) + log(1 + exp((B_NZONE1/mu ) * (nzone1_b-nzone1_a))) + log(1 + exp((B_NZONE2/mu) * (nzone2_sq-nzone2_a))) + log(1 + exp((B_NZONE2 / mu ) * (nzone2_b-nzone2_a))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_sq-nzone3_a))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_b-nzone3_a))) + log( 1 + exp((B_TOWNDOWN / mu ) * (towndown_sq-towndown_a))) + log(1 + exp((B_TOWNDOWN / mu ) * (towndown_b-towndown_a))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_sq-townmid_a))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_b-townmid_a))) + log(1 + exp((B_TOWNUP / mu ) * (townup_sq-townup_a))) + log(1 + exp((B_TOWNUP / mu ) * (townup_b-townup_a))) + log( 1 + exp((B_ECO / mu ) * (eco_sq-eco_a))) + log(1 + exp((B_ECO / mu ) * (eco_b-eco_a))) + log(1 + exp((B_REC / mu ) * (rec_sq-rec_a))) + log(1 + exp((B_REC / mu ) * (rec_b-rec_a))) + log(1 + exp((B_DRY / mu ) * (dry_sq-dry_a))) + log(1 + exp((B_DRY / mu ) * (dry_b-dry_a))) + log(1 + exp((B_TAX / mu ) * (tax_sq-tax_a))) + log(1 + exp((B_TAX / mu ) * (tax_b-tax_a))))   
           
RB  =scale* mu * (log( 1 + exp((B_NZONE1/mu) * (nzone1_sq-nzone1_b))) + log(1 + exp((B_NZONE1/mu ) * (nzone1_a-nzone1_b))) + log(1 + exp((B_NZONE2/mu) * (nzone2_sq-nzone2_b))) + log(1 + exp((B_NZONE2 / mu ) * (nzone2_a-nzone2_b))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_sq-nzone3_b))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_a-nzone3_b))) + log( 1 + exp((B_TOWNDOWN / mu ) * (towndown_sq-towndown_b))) + log(1 + exp((B_TOWNDOWN / mu ) * (towndown_a-towndown_b))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_sq-townmid_b))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_a-townmid_b))) + log(1 + exp((B_TOWNUP / mu ) * (townup_sq-townup_b))) + log(1 + exp((B_TOWNUP / mu ) * (townup_a-townup_b))) + log( 1 + exp((B_ECO / mu ) * (eco_sq-eco_b))) + log(1 + exp((B_ECO / mu ) * (eco_a-eco_b))) + log(1 + exp((B_REC / mu ) * (rec_sq-rec_b))) + log(1 + exp((B_REC / mu ) * (rec_a-rec_b))) + log(1 + exp((B_DRY / mu ) * (dry_sq-dry_b))) + log(1 + exp((B_DRY / mu ) * (dry_a-dry_b))) + log(1 + exp((B_TAX / mu ) * (tax_sq-tax_b))) + log(1 + exp((B_TAX / mu ) * (tax_a-tax_b))))   

In [11]:
# Associate utility functions with the numbering of alternatives
V = {1: -RSQ,
     2: -RA,
     3: -RB}

# Associate the availability conditions with the alternatives
# all alternatives are available for each individual
av = {1: 1,
      2: 1,
      3: 1}

In [12]:
# Definition of the model. This is the contribution of each observation to the log likelihood function.
logprob = models.loglogit(V,av,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "03logit_rrm_0428"

In [13]:
biogeme.createLogFile()

results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

biogeme.modelName = "03logit_rrm_0428"


print("Results=",results)

KeyboardInterrupt: 